# Neural network to classify if two samples are or not in the same gauge-orbit

In [2]:
import sys
sys.path.append('src/')
import tools as t
import numpy as np


In [3]:
# linear size of the system
L=5
Lx = L
Ly = L

# Number of sites
N = Lx*Ly

# to be commented
connectivity,getindex=t.compute_connectivity(Lx,Ly)

ncmax=int(0.5*len(connectivity))

# number of samples
NS=20000
# periodic boundary conditions
PBC=False

print("Linear size L=",L)
print("Number of samples NS=",NS)


Linear size L= 5
Number of samples NS= 20000


# Creation of the dataset

In [4]:
NS_GG=NS #number of samples of (gauge,gauge)
NS_GL=NS #number of samples of (gauge,not gauge)
NS_TOT=NS_GG + NS_GL

if (PBC):
    xdim = 2 * Lx + 1
    ydim = 2 * Ly + 1
else:
    xdim = 2 * Lx 
    ydim = 2 * Ly 
    
DataSet = np.zeros((NS_GG  + NS_GL, xdim, ydim, 2))
Labels = np.zeros((NS_GG + NS_GL))

for ns in range(NS_GG + NS_GL):

    if(ns<NS_GG): # we start with the couples Gauge-gauge

        #we create a random sample
        Chess = t.createSample_2D(connectivity, Lx,Ly)
        #we create a random orbit from it
        Orbit = t.getOrbit_2D(Chess,connectivity, Lx,Ly)

        if (PBC):
            DataSet[ns, :, :, 0] = t.write_PBC(Chess,Lx,Ly)
            DataSet[ns, :, :, 1] = t.write_PBC(Orbit)
        else:
            DataSet[ns, :, :, 0] = Chess
            DataSet[ns, :, :, 1] = Orbit

        Labels[ns]=0 # gauge-gauge

    else:  # we create couples gauge-not gauge

        transformation_type=np.random.randint(3)
        #we create a random sample
        Chess = t.createSample_2D(connectivity, Lx,Ly)

        if transformation_type==0: # we invert a random number of links
            #random number of links in between 1 and Lx*Ly (all plaquettes changed)
            nchanges=np.random.randint(1,int(0.25*len(connectivity)))
            q=float(nchanges/len(connectivity))

            #invert q of the total couplings randomly
            Trasf = t.getRandom_2D(Chess, connectivity, q, Lx,Ly)

        if transformation_type==1: # we invert a line
            Trasf = t.getLine_2D(Chess,connectivity, Lx,Ly)

        if transformation_type==2: # invert only 1-5 random links 
            nchanges=np.random.randint(1,5)
            q=float(nchanges/len(connectivity))
            Trasf = t.getRandom_2D(Chess, connectivity, q, Lx,Ly)	

        # We generate a random gauge orbit of this transformation (otherwise it would be 
        # too easy for the machine )
        Trasf = t.getOrbit_2D(Trasf,connectivity, Lx,Ly)

        if(PBC):
            DataSet[ns, :, :, 1] = t.write_PBC(Trasf,Lx,Ly)
            DataSet[ns, :, :, 0] = t.write_PBC(Chess,Lx,Ly)
        else:
            DataSet[ns, :, :, 1] = Trasf   
            DataSet[ns, :, :, 0] = Chess 

        Labels[ns]=1 # gauge - not gauge

# we reshuffle the data to mix the two kinds
permutation = np.random.permutation(Labels.shape[0])
DataSet = DataSet[permutation,:,:,:]
Labels = Labels[permutation]

# We split the data set in 0.8 for the training - 0.2 for the test
X_train,y_train,X_test,y_test = t.SplitSet(DataSet,Labels,0.8)
print(X_train.shape)
print(X_test.shape)


(32000, 10, 10, 2)
(8000, 10, 10, 2)


# Construction of the neural-network

In [19]:
import keras
from keras.layers import Input, Dense, Conv2D, Flatten, concatenate
from keras.models import Model
from keras import regularizers, metrics
from keras.optimizers import SGD, Adam,K
from keras.utils.np_utils import to_categorical

data_dim_x = xdim
data_dim_y = ydim

Strain = X_train.shape[0]
Stest = X_test.shape[0]

Nfilter=64
input_data = Input(shape=(data_dim_x,data_dim_y,2))

# one layer vertical slab CNN
vert_conv2D = Conv2D(Nfilter,(1,data_dim_y),activation='relu',strides=(1,1),padding='valid')(input_data)
vert_conv2D = Flatten()(vert_conv2D)

# one layer horizontal slab CNN
horiz_conv2D = Conv2D(Nfilter,(data_dim_x,1),activation='relu',strides=(1,1),padding='valid')(input_data)
horiz_conv2D = Flatten()(horiz_conv2D)

# one layer plaquettes CNN
plaq_conv2D = Conv2D(Nfilter,(3,3),activation='relu', padding='same',strides=(2,2))(input_data)
plaq_conv2D = Flatten()(plaq_conv2D)

# we concatenate all three results
x = concatenate([vert_conv2D, horiz_conv2D, plaq_conv2D])

# Dense layer
x = Dense(Nfilter,activation='relu')(x)

# Output
main_output = Dense(1,activation='sigmoid')(x)

DCNN = Model(input_data,main_output)

#funcionan
opt_sgd = SGD(lr=0.01,momentum=0.5,nesterov=True) 
opt_adam = Adam()

#batch size
b_size = 32


# Learning process

We didn't find a simple way to do the learning and a combination of optimizers were necessary to be able to fit the machine for all the values of $L$ and $N_S$ and to avoid overfitting. The learning process ends when the accuracy on the test set goes beyond 0.995. We show a configuration that worked for all the parameters used in the paper. Sometimes the learning process gets stuck in a minimum, and it's more efficient to restart the learning than to continue.

In [20]:
name_file_intantaneous="acceptancy_L"+str(Lx)+"_with_NS"+str(NS)+".txt"
file_out=open(name_file_intantaneous,"w")
file_out.write("# it Acc_train Acc_test"+"\n")
file_out.flush()

Acc=[]
Acc_val=[]



restart=False

acc=0
it=0



# we start with SGD optimizer until acc 
DCNN.compile(optimizer=opt_sgd,loss='binary_crossentropy', metrics=["accuracy"])
DCNN.summary()
while acc<0.995:
    
    hist=DCNN.fit(X_train.reshape(Strain,data_dim_x,data_dim_y,2),
                     y_train,epochs=1,
                         batch_size=b_size,
                         shuffle=True,
                         validation_data=(X_test.reshape(Stest,data_dim_x,data_dim_y,2),y_test))

    acc=hist.history['acc'][-1]
    acc_val=hist.history['val_acc'][-1]
    Acc.extend(hist.history['acc'])
    Acc_val.extend(hist.history['val_acc'])
    file_out.write(str(len(Acc))+" "+str(acc)+" "+str(acc_val)+"\n")
    file_out.flush()

    if it>0 and len(Acc)>5 and abs((Acc[-1]-Acc[-2])/Acc[-1])<0.01: # If accuracy gets stuck, we decrease the learning rate
        lr = K.get_value(DCNN.optimizer.lr)
        K.set_value(DCNN.optimizer.lr, lr*.9)
        print("lr changed to {}".format(lr*.9))
        it=0
        if lr < 1e-5:
            break
    if it>2 and acc>0.8:
        break

    it+=1
    
    
lr0 = K.get_value(DCNN.optimizer.lr) # we save the learning rate

if acc<0.995:
    #we change the optimizer to adam
    DCNN.compile(optimizer=opt_adam,loss='binary_crossentropy', metrics=["accuracy"])
    DCNN.summary()


it=0
while acc<0.995:
    
    hist=DCNN.fit(X_train.reshape(Strain,data_dim_x,data_dim_y,2),
             y_train,epochs=1,
             batch_size=b_size,
             shuffle=True,
             validation_data=(X_test.reshape(Stest,data_dim_x,data_dim_y,2),y_test))
    acc=hist.history['acc'][-1]
    acc_val=hist.history['val_acc'][-1]
    Acc.extend(hist.history['acc'])
    Acc_val.extend(hist.history['val_acc'])
    file_out.write(str(len(Acc))+" "+str(acc)+" "+str(acc_val)+"\n")
    file_out.flush()

    if it>2 and abs(Acc[-1]-Acc[-2])<0.001 and Acc[-2]<Acc[-1]:# if the accuracy gets stuck we get out
        break

    
    
    it+=1

    
    
while acc<0.995:

    lr0=(1-acc)*0.03
    print("lr started with {}".format(lr0))
    
    #we come back to the SGD optimizer
    DCNN.compile(optimizer=opt_sgd,loss='binary_crossentropy', metrics=["accuracy"])
    K.set_value(DCNN.optimizer.lr, lr0)
    it=0
    while acc<0.995:   
        hist=DCNN.fit(X_train.reshape(Strain,data_dim_x,data_dim_y,2),
                         y_train,epochs=1,
                             batch_size=b_size,
                             shuffle=True,
                             validation_data=(X_test.reshape(Stest,data_dim_x,data_dim_y,2),y_test))

        acc=hist.history['acc'][-1]
        acc_val=hist.history['val_acc'][-1]
        Acc.extend(hist.history['acc'])
        Acc_val.extend(hist.history['val_acc'])
        file_out.write(str(len(Acc))+" "+str(acc)+" "+str(acc_val)+"\n")
        file_out.flush()

        if it>3 and len(Acc)>5 and abs((Acc[-1]-Acc[-2]))<0.005:
            lr = K.get_value(DCNN.optimizer.lr)
            K.set_value(DCNN.optimizer.lr, lr*.9)
            print("lr changed to {}".format(lr*.9))

            it=0
            if lr < 1e-3:
                break

        if it>1 and abs(Acc[-1]-Acc[-2])<0.0001  and Acc[-2]<Acc[-1]:
            break

        it+=1
        
    while acc<0.995:
        #back to Adam optimizer
        DCNN.compile(optimizer=opt_adam,loss='binary_crossentropy', metrics=["accuracy"])
        DCNN.summary()

        it=0
        hist=DCNN.fit(X_train.reshape(Strain,data_dim_x,data_dim_y,2),
                 y_train,epochs=1,
                 batch_size=b_size,
                 shuffle=True,
                 validation_data=(X_test.reshape(Stest,data_dim_x,data_dim_y,2),y_test))
        acc=hist.history['acc'][-1]
        acc_val=hist.history['val_acc'][-1]
        Acc.extend(hist.history['acc'])
        Acc_val.extend(hist.history['val_acc'])
        file_out.write(str(len(Acc))+" "+str(acc)+" "+str(acc_val)+"\n")
        file_out.flush()

        
        if it>2 and abs(Acc[-1]-Acc[-2])<0.01 :
            break
        it+=1

        if acc<0.6:
            restart=True
            break

    if len(Acc)>100:
        if acc<0.99:
            restart=True
        break

if restart:
    print("Re-run the program (definition of the DCNN and fit), it got stuck!")
else:
    print("Learning FINISHED, final acc_test ",acc_val," maximum one ",max(Acc_val))
file_out.close()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 10, 2)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 10, 1, 64)    1344        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 1, 10, 64)    1344        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 5, 5, 64)     1216        input_3[0][0]                    
__________________________________________________________________________________________________
flatten_7 

32000/32000 [==============================] - 9s 282us/step - loss: 0.1152 - acc: 0.9561 - val_loss: 0.1413 - val_acc: 0.9477
Train on 32000 samples, validate on 8000 samples
Epoch 1/1
32000/32000 [==============================] - 9s 277us/step - loss: 0.1011 - acc: 0.9626 - val_loss: 0.1602 - val_acc: 0.9351
Train on 32000 samples, validate on 8000 samples
Epoch 1/1
32000/32000 [==============================] - 9s 282us/step - loss: 0.0909 - acc: 0.9667 - val_loss: 0.1682 - val_acc: 0.9326
Train on 32000 samples, validate on 8000 samples
Epoch 1/1
32000/32000 [==============================] - 9s 277us/step - loss: 0.0828 - acc: 0.9682 - val_loss: 0.2327 - val_acc: 0.9331
Train on 32000 samples, validate on 8000 samples
Epoch 1/1
32000/32000 [==============================] - 9s 278us/step - loss: 0.0746 - acc: 0.9721 - val_loss: 0.2921 - val_acc: 0.8828
Train on 32000 samples, validate on 8000 samples
Epoch 1/1
32000/32000 [==============================] - 9s 279us/step - loss: 0